In [0]:
df_oct = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/workspace/ecom/ecom_data/2019-Oct.csv")
)

In [0]:
df_nov = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/workspace/ecom/ecom_data/2019-Nov.csv")
)


In [0]:
df_all = df_oct.unionByName(df_nov)
df_all.show()

In [0]:
df_all.display()

In [0]:
df_all.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecom_orders")

In [0]:
%sql
SELECT COUNT(*) FROM ecom_orders;

In [0]:
for i in range(3):
    (
        df_nov
        .repartition(5)   ### force multiple small partitions
        .write
        .format("delta")
        .mode("append")
        .saveAsTable("ecom_orders")
    )

In [0]:
%sql
OPTIMIZE ecom_orders;

In [0]:
%sql
DESCRIBE DETAIL ecom_orders;